# Clamped Reissner-Mindlin plate under uniform load using TDNNS finite element method

This demo program solves the out-of-plane Reissner-Mindlin equations on the
unit square with uniform transverse loading with fully clamped boundary
conditions using the Tangential-Displacement-Normal-Normal-Stress (TDNNS)
method described in Pechstein and Schöberl [ref]. Instead of writing the 
Lagrangian for the energy and then differentiating we write the minimisation
directly. We follow the notation of Pechstein and Schöberl [ref] throughout.

It is assumed the reader understands most of the basic functionality
of the new FEniCSx Project.

This demo illustrates how to:

We begin by importing the necessary functionality from DOLFINx, UFL and PETSc.

In [1]:
import dolfinx
import numpy as np
import ufl
from dolfinx import Constant, DirichletBC, Function, FunctionSpace, UnitSquareMesh
from dolfinx.cpp.mesh import CellType
from dolfinx.fem import (
    apply_lifting,
    assemble_matrix,
    assemble_vector,
    locate_dofs_topological,
    set_bc,
)
from dolfinx.io import XDMFFile
from dolfinx.mesh import locate_entities_boundary
from mpi4py import MPI
from petsc4py import PETSc
from ufl import (
    FacetNormal,
    FiniteElement,
    Identity,
    MixedElement,
    VectorElement,
    dot,
    ds,
    dS,
    dx,
    grad,
    inner,
    jump,
    split,
    sym,
    tr,
)

We then create a two-dimensional mesh of the mid-plane of the plate $\Omega = [0, 1] \times [0, 1]$. `GhostMode.shared_facet` is required.

In [2]:
mesh = UnitSquareMesh(
    MPI.COMM_WORLD, 32, 32, CellType.triangle, dolfinx.cpp.mesh.GhostMode.shared_facet
)

The first-order TDNNS element for the Reissner-Mindlin plate problem consists of:

- the first-order tensor-valued $H(\mathrm{div} \; \mathrm{div})$-conforming rotated Regge element for the bending moments $M \in \mathrm{SREG}_1$,
- the first-order vector-valued Nedéléc element of the second-kind for the rotation field $\theta \in
  \mathrm{NED}_1^2$ and,
- a second-order scalar-valued Lagrange element for the transverse
  displacement field $w \in \mathrm{CG}_2$.

The final element definition is

In [3]:
U_el = MixedElement(
    [
        FiniteElement("Regge", ufl.triangle, 1),
        FiniteElement("N2curl", ufl.triangle, 1),
        FiniteElement("Lagrange", ufl.triangle, 2),
    ]
)
U = FunctionSpace(mesh, U_el)

m, theta, w = ufl.TrialFunctions(U)
tau, eta, v = ufl.TestFunctions(U)

We assume constant material parameters; Young's modulus $E$, Poisson's ratio $\nu$, shear-correction factor $\kappa$, and thickness
$t$.

In [4]:
E = 10920.0
nu = 0.3
kappa = 5.0 / 6.0
t = 0.001

In [5]:
def A_b(m):
    """Bending compliance"""
    return (12.0 / E) * ((1 + nu) * m - nu * tr(m) * Identity(2))

In [8]:
def b(tau, eta):
    """Discrete duality inner product"""
    n = FacetNormal(mesh)
    return (
        inner(tau, sym(grad(eta))) * dx
        - dot(dot(tau("+"), n("+")), n("+")) * jump(dot(eta, n)) * dS
        - dot(dot(tau, n), n) * dot(eta, n) * ds
    )


mu = kappa * E / (2.0 * (1.0 + nu))

a = (
    inner(A_b(m), tau) * dx
    + b(tau, theta)
    + b(m, eta)
    - mu * t ** -2 * inner(grad(w) - theta, grad(v) - eta) * dx
)
L = -inner(1.0, v) * dx

# TODO: Check boundary conditions
u0 = Function(U)
u0.vector.set(0.0)
facets = locate_entities_boundary(mesh, 1, lambda x: np.ones(x.shape[1], dtype=bool))
dofs0 = locate_dofs_topological(U, 1, facets)
bcs = [DirichletBC(u0, dofs0)]

In the following we use standard from `dolfinx` to apply boundary conditions, assemble, solve and output the solution.

In [7]:
A = dolfinx.fem.assemble_matrix(a, bcs=bcs)
A.assemble()

b = dolfinx.fem.assemble_vector(L)
dolfinx.fem.apply_lifting(b, [a], [bcs])
b.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)
dolfinx.fem.set_bc(b, bcs)

ksp = PETSc.KSP().create(MPI.COMM_WORLD)

pc = ksp.getPC()
pc.setType("lu")
pc.setFactorSolverType("mumps")

u_ = Function(U)
ksp.setOperators(A)
ksp.setType("preonly")
ksp.setFromOptions()
ksp.solve(b, u_.vector)

bb_tree = dolfinx.cpp.geometry.BoundingBoxTree(mesh, 2)
point = np.array([0.5, 0.5, 0.0], dtype=np.float64)
cell_candidates = dolfinx.cpp.geometry.compute_collisions_point(bb_tree, point)
cell = dolfinx.cpp.geometry.select_colliding_cells(mesh, cell_candidates, point, 1)

m, theta, w = u_.split()

if len(cell) > 0:
    value = w.eval(point, cell)
    print(value[0])
    # NOTE: FEniCS-Shells (old dolfin) `demo/documented/reissner-mindlin-clamped`
    # gives 1.28506469462e-06 on a 32 x 32 mesh and 1.2703580973e-06 on a 64 x 64
    # mesh.
    def test_center_displacement():
        assert np.isclose(value[0], 1.285e-6, atol=1e-3, rtol=1e-3)


with XDMFFile(MPI.COMM_WORLD, "w.xdmf", "w") as f:
    f.write_mesh(mesh)
    f.write_function(w)

with XDMFFile(MPI.COMM_WORLD, "theta.xdmf", "w") as f:
    f.write_mesh(mesh)
    f.write_function(theta)

2.630523260437713e-09


## Appendix